# Boat Control with Model Predictive Control

This notebook provides a template for setting up and running a boat control simulation using the Model Predictive Control (MPC) framework with `do-mpc`. 
Fill in the missing logic for the Boat class methods, MPC model setup, and the control loop as per your application's requirements.


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import do_mpc
from boat_simulation import THETA, SPEED

import casadi as ca

speed_interp = ca.interpolant('speed_interp', 'linear', [THETA.tolist()], SPEED.tolist())


mass = 960  # Mass of the boat in kg
drag_coefficient = 0.003  # Drag coefficient of the boat
reference_area = 7.48  # Estimated wetted surface area in square meters
inertia_factor = 0.2  # Factor to reduce inertia
water_density = 1025  # Density of salt water in kg/m^3
reference_area = 7.48 # Estimated wetted surface area in square meters
drag_force_constant = 0.5 * water_density * drag_coefficient * reference_area


def wrap_phase(phase):
    two_pi = 2 * ca.pi
    return ca.fmod(ca.fmod(phase, two_pi) + two_pi, two_pi)

def predict_speed(theta, dead_zone_angle=np.pi/6, wrap=True):
    theta = wrap_phase(theta) if wrap else theta
    speed = speed_interp(theta)
    return ca.if_else(ca.fabs(theta - ca.pi) < dead_zone_angle, 0, speed)

def adjust_speed(speed, predicted_speed):
    propulsive_force = drag_force_constant * predicted_speed ** 2
    drag_force = drag_force_constant * speed ** 2
    net_force = propulsive_force - drag_force
    acceleration = net_force / mass
    return ca.fmax(speed + acceleration, 0)

def distance_to_waypoint(x, y, waypoint):
    return ca.sqrt((x - waypoint[0])**2 + (y - waypoint[1])**2)

def setup_mpc_model():
    model = do_mpc.model.Model('continuous', symvar_type='MX')

    # Define states and controls
    # Example: x, y for position; theta for orientation; v for velocity
    # u_rudder, u_sail for control inputs
    x = model.set_variable(var_type='_x', var_name='x', shape=(1,1))
    y = model.set_variable(var_type='_x', var_name='y', shape=(1,1))
    speed = model.set_variable(var_type='_x', var_name='speed', shape=(1,1))
    heading = model.set_variable(var_type='_x', var_name='heading', shape=(1,1))

    predicted_speed = model.set_variable(var_type='_z', var_name='predicted_speed', shape=(1,1))

    # ... other states ...

    target_heading = model.set_variable(var_type='_u', var_name='target_heading')
    # ... other controls ...

    # Define the system dynamics
    # Example: dx = v * cos(theta), dy = v * sin(theta), ...
    model.set_rhs('heading', wrap_phase((target_heading - heading) * (1 - inertia_factor)))

    model.set_alg('predicted_speed', predict_speed(heading))

    model.set_rhs('speed', adjust_speed(speed, predicted_speed))
    model.set_rhs('x', x + speed * ca.sin(heading))
    model.set_rhs('y', y + speed * ca.cos(heading))


    model.setup()


    return model

# Define waypoints
waypoints = np.array([[0, 250], [0, 0], [0, -250]])
current_waypoint_index = 0
waypoint_threshold = 10

# Setup MPC
mpc_model = setup_mpc_model()
mpc = do_mpc.controller.MPC(mpc_model)

stage_cost = distance_to_waypoint(mpc_model.x['x'], mpc_model.x['y'], waypoints[current_waypoint_index])
mpc.set_objective(mterm=stage_cost, lterm=stage_cost)  # Terminal and stage cost
mpc.set_param(n_horizon=20,  # Prediction horizon
              n_robust=1,    # Robust horizon
              t_step=1.0)    # Time step
mpc.setup()

x0 = np.array([0, 0, 0, 0]).reshape(-1,1)

mpc.x0 = x0
mpc.set_initial_guess()

estimator = do_mpc.estimator.StateFeedback(mpc_model)
estimator.x0 = x0

params_simulator = {
    # Note: cvode doesn't support DAE systems.
    'integration_tool': 'idas',
    'abstol': 1e-8,
    'reltol': 1e-8,
    't_step': 0.04
}
simulator = do_mpc.simulator.Simulator(mpc_model)
simulator.set_param(**params_simulator)
simulator.setup()


simulated_x_positions = []
simulated_y_positions = []
for timestep in range(500):
    # Solve MPC problem
    mpc.solve()

    simulated_x_positions.append(mpc.data['_x', 'x'])
    simulated_x_positions.append(mpc.data['_x', 'y'])

    # Apply control inputs to the boat
    u0 = mpc.make_step(x0)
    y_next = simulator.make_step(u0)
    x0 = estimator.make_step(y_next)

    # Check if current waypoint is reached
    if distance_to_waypoint(mpc_model.x['x'], mpc_model.x['y'], waypoints[current_waypoint_index]) < waypoint_threshold:
        current_waypoint_index += 1
        if current_waypoint_index >= len(waypoints):
            break  # All waypoints reached

        # Update the MPC model with the new waypoint
        stage_cost = distance_to_waypoint(mpc_model.x['x'], mpc_model.x['y'], waypoints[current_waypoint_index])
        mpc.set_objective(mterm=stage_cost, lterm=stage_cost) # Update the objective function
        mpc.setup()

plt.plot(simulated_x_positions, simulated_y_positions, label='Trajectory')
plt.plot(waypoints[:, 0], waypoints[:, 1], 'ro', label='Waypoints')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Boat Trajectory')
plt.legend()
plt.show()

RuntimeError: .../casadi/core/function_internal.cpp:146: Error calling IdasInterface::init for 'simulator':
.../casadi/core/integrator.cpp:646: Assertion "!sp_jac_dae_.is_singular()" failed:
Jacobian of the forward problem is structurally rank-deficient. sprank(J)=4<5